In [1]:
import torch
import torchvision
import onlinehd
import torchvision.datasets as datasets
import numpy as np
import sklearn
from sklearn import *
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# open/read the .csv dataset with panda
coll = pd.read_csv("./data/data.csv") 

In [3]:
# make categorical variables numeric
coll["type_school"]= coll["type_school"].replace("Academic", 0)
coll["type_school"]= coll["type_school"].replace("Vocational", 1)
coll["school_accreditation"]= coll["school_accreditation"].replace("A", 0)
coll["school_accreditation"]= coll["school_accreditation"].replace("B", 1)
coll["gender"]= coll["gender"].replace("Male", 1)
coll["gender"]= coll["gender"].replace("Female", 0)
coll["interest"]= coll["interest"].replace("Not Interested", -2)
coll["interest"]= coll["interest"].replace("Less Interested", -1)
coll["interest"]= coll["interest"].replace("Uncertain", 0)
coll["interest"]= coll["interest"].replace("Quiet Interested", 1)
coll["interest"]= coll["interest"].replace("Very Interested", 2)
coll["residence"]= coll["residence"].replace("Urban", 1)
coll["residence"]= coll["residence"].replace("Rural", 0)
coll["parent_was_in_college"]= coll["parent_was_in_college"].replace(True, 1)
coll["parent_was_in_college"]= coll["parent_was_in_college"].replace(False, 0)
coll["in_college"]= coll["in_college"].replace("TRUE", 1)
coll["in_college"]= coll["in_college"].replace("FALSE", 0)

In [4]:
# make it a numpy array so numpy can read it
coll = np.array(coll)

# first 8 columns of matrix are considered parameters
# the last one is considered to be the class
X = np.array(coll[:, :10])
Y = np.array(coll[:, 10])

In [5]:
X, X_test, Y, Y_test = sklearn.model_selection.train_test_split(X, Y)
print(type(X[0]), X[0])
X = torch.Tensor(X.astype('float64'))
Y = torch.Tensor(Y.astype('int64'))
X_test = torch.Tensor(X_test.astype('float64'))
Y_test = torch.Tensor(Y_test.astype('int64'))

print("X train shape", X.shape, "Y train shape", Y.shape)
print("X test shape", X_test.shape, "Y Test shape", Y_test.shape)

<class 'numpy.ndarray'> [1 1 0 0 0 54 3700000 99.6 83.5 1]
X train shape torch.Size([750, 10]) Y train shape torch.Size([750])
X test shape torch.Size([250, 10]) Y Test shape torch.Size([250])


In [6]:
from fhrr import FHRR_Model

In [7]:
classes = len(np.unique(Y))
features = X.shape[1]
print("Number of classes", classes, '\nNumber of features', features)

Number of classes 2 
Number of features 10


In [8]:
dim = 1000

model = FHRR_Model(classes, features, dim)
# model.set_train_set(X, Y)
model.encode_set(X, Y)

X train shape (750, 1000)


In [9]:
model.train()

preds = [model.predict(model.encode(x)) for x in X_test]
acc = sklearn.metrics.accuracy_score(Y_test, preds)
print(acc)

Training class hypervectors...
 [[ 17.4683487  -77.58674303j 355.72574122 -98.36097632j
  262.01784034-177.7034103j  ... 232.08084628-186.74722424j
  327.75455183-134.35662119j 223.91074178+188.31543898j]
 [-13.84187753 -80.83644954j 338.58336388-102.48080779j
  240.95052475-185.14648508j ... 209.75962519-194.56909751j
  309.44060636-139.98412365j 201.24731735+196.20299664j]]
0.468


In [10]:
model.retrain(100)

preds = [model.predict(model.encode(x)) for x in X_test]
acc = sklearn.metrics.accuracy_score(Y_test, preds)
print(acc)

0.468
